In [4]:
import sys
import os
import math
import Stemmer
import time
from nltk.corpus import stopwords

In [8]:
stop_words=set(stopwords.words('english'))
my_stemmer = Stemmer.Stemmer('english')
TITLE = 0
BODY = 1
INFOBOX = 2
CATEGORY = 3
REFERENCE = 4
LINKS = 5
TYPES = 6
TYPE_LIST = ['t', 'b', 'i', 'c', 'r', 'e']

In [ ]:
'''
This finds the term using binary search and returns its posting list
'''
def find_term(term):
    posting = ""
    return -1, posting

In [10]:
'''
This returns list of document_ID which contain 'term' in 'field'
'''
def handle_field_query(term, field):
    index, posting = find_term(term)
    result = []
    if index == -1:
        return result
    posting = posting.split()
    for i in range(TYPES):
        if field == i:
            for data in posting:
                if TYPE_LIST[i] in data:
                    docID = ''
                    for char in data:
                        if char >= 'a' and char <= 'z':
                            break
                        else:
                            docID += char
                    result.append(docID)
            break
    return result

In [12]:
'''
This returns list of document_ID which contains 'term' anywhere in it
'''
def handle_simple_query(term):
    index, posting = find_term(term)
    result = []
    if index == -1:
        return result
    posting = posting.split()
    for data in posting:
        docID = ''
        for char in data:
            if char >= 'a' and char <= 'z':
                break
            else:
                docID += char
        result.append(docID)
    return result

In [ ]:
def main():
    field_queries = ['t:', 'b:', 'i:', 'c:', 'r:', 'e:']
    field_query = [0]*TYPES
    
    '''
    we assume queries will follow certain rules
    1. If a field query is asked than it will be of the form 't:sachin india great i:2019 c:england'
    2. Which means this is not allowed 't:sachin t:india c:2019 i:great c:help'
    
    Maybe we can parse these, YES WE CAN!!
    '''
    while True:
        query = input("ASK : ")
        start_time = time.time()
        query = query.lower().strip()
        if query == 'exit':
            break
        terms = query.split()
        query_terms = []
        for term in query_terms:
            term = term.lower().strip()
            if term not in stop_words
                term = my_stemmer.stemWord(term)
                query_terms.append(term)
        if len(query_terms) == 0:
            print('This is not a good Query, Ask something meaningful')
            continue
        
        
        